In [4]:
!pip install -q pyngrok flask scikit-learn joblib nltk pandas pyngrok
!ngrok config add-authtoken 2vPPRL7gp7ohQwwxp048TkFhJpL_7jzv85t4JpCWer6fPsTGr
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import joblib
from pyngrok import ngrok
from flask import Flask, jsonify, render_template, request

from google.colab import drive
drive.mount('/content/drive')

# read data set
df=pd.read_csv('drive/MyDrive/MyProjects/IMDB Dataset.csv')
print(df.head())

nltk.download('stopwords')
nltk.download('wordnet')

#clean data(review column)
def clean_review(review):
    review=re.sub(r'\W',' ',review)# remove special characters
    review=review.lower()# all character to lowercase
    words=review.split()#split the words from text
    words=[word for word in words if word not in stopwords.words('english')]#a list of words
    lemmatizer=WordNetLemmatizer()#limit the words into their base like: "Running -> Run"
    words=[lemmatizer.lemmatize(word) for word in words]# list of words converted into base.
    return ' '.join(words)#after cleaning join all the words.

print("Cleaning review data...")
df['review']=df['review'].fillna("") #Ensure no nan values
df['clean_review']=df['review'][:1000].apply(clean_review)#sliced dataset
df['clean_review']=df['clean_review'].fillna("")
print("Review Data cleaned Successfulluy!")


#convert text into numercal form
vectorizer=TfidfVectorizer(max_features=500) #(term frequency-inverse document frequency)
X=vectorizer.fit_transform(df['clean_review'])
Y=df['sentiment'].map({'positive':1,'negative':0}) # convert labels to binary values

#train-test split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)# Split the data set into 80% training & 20% testing.

#Train the model
model=MultinomialNB()# create the model
model.fit(X_train,Y_train) # Using training model to train the data


#Evaluate the model
Y_pred=model.predict(X_test)
print("Accurace: ",accuracy_score(Y_test,Y_pred))

#Flask app
app=Flask(__name__, template_folder='/content/drive/MyDrive/MyProjects/templates')

# Save model and vectorizer
joblib.dump(model, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

@app.route('/')
def home():
    return render_template("index3.html")

@app.route('/predict',methods=['POST'])
def predict():
    review=request.form['review']
    # processed=clean_review(review)
    # processed=[processed]
    vect=vectorizer.transform([review])
    prediction=model.predict(vect)[0]
    sentiment="positive" if prediction==1 else "negative"
    return render_template("index3.html",prediction=f"The Review is: {sentiment}")

public_url = ngrok.connect(5000)
print(f" * ngrok tunnel running at: {public_url}")
app.run(port=5000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
Cleaning review data...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Review Data cleaned Successfulluy!
Accurace:  0.5026
 * ngrok tunnel running at: NgrokTunnel: "https://d4b3-34-86-56-75.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 17:55:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 17:55:40] "GET /favicon.ico HTTP/1.1" 404 -
